In [ ]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langchain-google-genai  langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install -qU pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.3 MB/s eta 0:00:00


In [1]:
import os
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [4]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langchain-google-genai  langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.2 MB/s eta 0:00:00


In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings.embed_query("Hello world")

[0.04703257977962494,
 -0.04019005596637726,
 -0.02902696467936039,
 -0.026809632778167725,
 0.01892058178782463,
 -8.365760731976479e-05,
 0.03111639805138111,
 -0.019520554691553116,
 0.011491335928440094,
 0.009625785052776337,
 0.04571185261011124,
 0.051709506660699844,
 -0.007854084484279156,
 -0.07627559453248978,
 -0.000736518413759768,
 -0.022592447698116302,
 0.011496763676404953,
 -0.0076109664514660835,
 0.006400744896382093,
 -0.0036826308351010084,
 -8.639021689305082e-05,
 0.007910552434623241,
 -0.031401973217725754,
 -0.027668770402669907,
 0.013148307800292969,
 0.005762958899140358,
 -0.0022430468816310167,
 -0.07029420882463455,
 0.007011014502495527,
 0.07013051956892014,
 -0.047634560614824295,
 0.008311820216476917,
 -0.06021169573068619,
 0.01643129624426365,
 0.042709145694971085,
 -0.047674257308244705,
 0.0342608243227005,
 0.021967323496937752,
 -0.007065100595355034,
 0.00032590707996860147,
 0.013825682923197746,
 -0.08921292424201965,
 -0.0340406857430934

In [7]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec

index_name = "quickstart4"
pc = Pinecone(api_key='pcsk_3FqWhm_UWGXupcg3cFf1UfQPFwRCg9FMmBQahC4qUvpA5wJCzJNdWSWx2VKjwon1x5d9gZ')

pc.create_index(
      name=index_name,
      dimension=768, # Replace with your model dimensions
      metric="cosine", # Replace with your model metric
      spec=ServerlessSpec(
          cloud="aws",
          region="us-east-1"
      )
  )

index = pc.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

In [11]:
from google.colab import files
uploaded = files.upload()

Saving documents.txt to documents.txt


In [12]:
import os
print(os.listdir("."))


['.config', 'documents.txt', 'sample_data']


In [13]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader("documents.txt")  # Replace with your file
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(docs)

[Document(metadata={'source': 'documents.txt'}, page_content='LangChain provides an extensive toolkit for building language model applications. \nIt offers modules for different components, such as prompt templates, chains, and memory. \nLangChain helps in integrating models with various data sources and handling conversational AI tasks effectively. \nYou can customize the toolkit to suit specific use cases, whether for text generation, chatbots, or data analysis.'), Document(metadata={'source': 'documents.txt'}, page_content='RecursiveCharacterTextSplitter is a useful utility in LangChain for splitting large text documents into manageable chunks.\nIt ensures that each chunk size remains under a specified limit, preventing text truncation and enabling better context handling for language models.\nBy using RecursiveCharacterTextSplitter, you can control chunk overlap and size, ensuring better performance and accuracy.')]


In [14]:
!pip install tqdm

In [22]:
from tqdm import tqdm
vectors=[]
for doc in tqdm(docs):
    vector = embeddings.embed_query(doc.page_content)
    doc_id = str(hash(doc.page_content))
    index.upsert(vectors=[(doc_id, vector, {'text': doc.page_content})])

100%|██████████| 2/2 [00:00<00:00,  3.11it/s]


In [23]:
from langchain.vectorstores import Pinecone

retriever=  Pinecone(index=index, embedding=embeddings, text_key="text")
retriever = vector_store.as_retriever()

In [24]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=retriever
)

In [25]:
query = "What is LangChain used for?"
response = qa_chain.run(query)
print(response)

LangChain is used for building applications powered by large language models (LLMs).  It provides tools and utilities to streamline the development process, including features like the `RecursiveCharacterTextSplitter`.  This splitter helps manage the input to LLMs by breaking down large text documents into smaller, manageable chunks, preventing issues caused by exceeding context windows and improving the accuracy and performance of LLM-based applications.  This is crucial for tasks like text generation, chatbots, question answering, and data analysis using LLMs.

